In [87]:
import numpy as np
import pandas as pd
import xgboost as xgb
import sklearn
import sklearn.tree
import sklearn.ensemble
import matplotlib.pyplot as plt
from sklearn.cross_validation import cross_val_score
from sklearn import preprocessing as pp
%matplotlib inline

In [88]:
data_train_read = pd.read_csv("train.csv")
data_test_read = pd.read_csv("test.csv")

In [89]:
data_train = data_train_read.copy()

In [90]:
data_test = data_test_read.copy()

In [91]:
X_train = data_train[data_train.columns[1:len(data_train.columns) - 1]]
y_train = data_train['TARGET']

In [92]:
X_test = data_test[data_test.columns[1:]]

# xgboost classifier


In [104]:
X_train.columns = [str(i) for i in np.arange(0, len(X_train.columns))]
X_test.columns = [str(i) for i in np.arange(0, len(X_test.columns))]
y_train.columns = ['0']

In [105]:
dtrain = xgb.DMatrix(X_train, label=y_train)

In [106]:
params = {}  
params['objective']           = "reg:linear" 
params['booster']             = "gbtree"
params['eval_metric']         = "auc"
params['eta']                 = 0.02
params['max_depth']           = 5
params['subsample']           = 0.681
params['colsample_bytree']    = 0.7

In [17]:
score = xgb.cv(params, dtrain, num_boost_round=571)
# xgboost.cv(params, dtrain, num_boost_round=10, nfold=3, stratified=False, folds=None, 
#            metrics=(), obj=None, feval=None, maximize=False, early_stopping_rounds=None, 
#            fpreproc=None, as_pandas=True, verbose_eval=None, show_stdv=True, seed=0)
score

,test-auc-mean,test-auc-std,train-auc-mean,train-auc-std
0,9.805831,4.442234,8.755162,2.298736
1,9.967538,4.495298,8.877995,2.214876
2,10.183652,4.610677,9.031965,2.110804
3,10.227003,4.509923,9.187857,2.258812
4,10.229395,4.573144,9.165992,2.158020
5,10.269232,4.644060,9.169646,2.103848
6,10.276564,4.667309,9.166017,2.092947
7,10.311194,4.664196,9.200924,2.124203
8,10.322940,4.624585,9.222780,2.164784
9,10.348309,4.635772,9.262578,2.196472


# commit

In [ ]:
bst = xgb.train(params=list(params.items()), dtrain=dtrain, num_boost_round=571)
bst.save_model('testmodel01.model')

In [ ]:
X_test.head(1)

In [ ]:
X_train.head(1)

In [45]:
dtest = xgb.DMatrix(X_test)
ypred = pd.DataFrame(bst.predict(dtest))
ypred.describe()

,0
count,76020.000000
mean,0.957662
std,1.502611
min,-0.908641
25%,0.116229
50%,0.406702
75%,0.990216
max,24.614553


In [46]:
ypred.columns = ['ans']

In [54]:
ypred[ypred['ans']>.99]=1
ypred[ypred['ans']<0.01]=0

In [56]:
ypred.describe()

,ans
count,76020.000000
mean,0.482383
std,0.384739
min,0.000000
25%,0.116229
50%,0.406702
75%,1.000000
max,1.000000


In [39]:
min_max_scaler = pp.MinMaxScaler()
ypred = pd.DataFrame(min_max_scaler.fit_transform(ypred))

In [44]:
ypred.describe()

,0
count,76020.000000
mean,0.073122
std,0.058873
min,0.000000
25%,0.040154
50%,0.051535
75%,0.074397
max,1.000000


In [57]:
ans = pd.DataFrame(index=data_test['ID'].values, data={'TARGET': ypred['ans'].values})
ans.to_csv('kek.csv')

ValueError: Shape of passed values is (1, 76020), indices imply (1, 75818)

In [ ]:
# 0.838212 - удалили дуб/юник
# 

# deleted dup/unique w\o scale 0.834372941429
# ^ with scale 0.827967473914 -> no scaling
todo: 
20 классифаеров с 1,0 пополам
кнн с весами откуда-нибудь
умное удаление дубликатов

# normalize

In [144]:
min_max_scaler = pp.MinMaxScaler()
X_train = pd.DataFrame(min_max_scaler.fit_transform(X_train))

In [93]:
newX_train = X_train.copy()

In [94]:
# remove constant columns
colsToRemoveConst = []
for col in newX_train.columns:
    if newX_train[col].std() == 0:
        colsToRemoveConst.append(col)

print "constant", len(colsToRemoveConst)

newX_train.drop(colsToRemoveConst, axis=1, inplace=True)

constant 34


In [97]:
X_test.drop(colsToRemoveConst, axis=1, inplace=True)

/Users/lokotochek/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [95]:
## @TODO:
## ебать макеенкова в носок
## если есть 2 колонки в тесте, значения не совпадают, но они в дубликатах, 
## то удаляю остальные, а в ост - themostfrequent

# remove duplicate columns
colsToRemove = []
columns = newX_train.columns
for i in range(len(columns)-1):
    v = newX_train[columns[i]].values
    for j in range(i+1,len(columns)):
        if np.array_equal(v, newX_train[columns[j]].values):
            colsToRemove.append(columns[j])
            
print "duplicate", pd.Series(colsToRemove).nunique()
newX_train.drop(colsToRemove, axis=1, inplace=True)

duplicate 29


In [98]:
X_test.drop(colsToRemove, axis=1, inplace=True)

/Users/lokotochek/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [99]:
X_train = newX_train

In [100]:
X_train = pd.DataFrame(X_train)
X_train.describe()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
count,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,...,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04
mean,-1523.199277,33.212865,86.208265,72.363067,119.529632,3.559130,6.472698,0.412946,0.567352,3.160715,...,0.186630,7.935824,1.365146,12.215580,8.784074,31.505324,1.858575,76.026165,56.614351,1.172358e+05
std,39033.462364,12.956486,1614.757313,339.315831,546.266294,93.155749,153.737066,30.604864,36.513513,95.268204,...,31.879418,455.887218,113.959637,783.207399,538.439211,2013.125393,147.786584,4040.337842,2852.579397,1.826646e+05
min,-999999.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.163750e+03
25%,2.000000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.787061e+04
50%,2.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.064092e+05
75%,2.000000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.187563e+05
max,238.000000,105.000000,210000.000000,12888.030000,21024.810000,8237.820000,11073.570000,6600.000000,6600.000000,8237.820000,...,7331.340000,50003.880000,20385.720000,138831.630000,91778.730000,438329.220000,24650.010000,681462.900000,397884.300000,2.203474e+07


In [101]:
gamma = 1.*len(y_train[y_train!=0])/(1.*len(y_train[y_train==0]))
gamma

0.04119870706185284

In [102]:
y_train[y_train == 1] = y_train[y_train == 1]/gamma

/Users/lokotochek/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [103]:
y_train.describe()

count    76020.000000
mean         0.960431
std          4.731812
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         24.272606
Name: TARGET, dtype: float64

In [ ]:
y_train = y_train*gamma